In [14]:
import pandas as pd
import numpy as np
price = pd.read_csv("Data.csv")

date_columns = price.columns[5:]

print(price.shape)
price.head()

(894, 290)


,RegionID,SizeRank,RegionName,RegionType,StateName,1/31/2000,2/29/2000,3/31/2000,4/30/2000,5/31/2000,...,12/31/2022,1/31/2023,2/28/2023,3/31/2023,4/30/2023,5/31/2023,6/30/2023,7/31/2023,8/31/2023,9/30/2023
0,394913,1,"New York, NY",msa,NY,214314.5418,215225.2576,216144.4084,218006.9635,219935.7038,...,602603.0589,601790.7385,600445.3576,600758.0745,602748.9688,606745.1232,610879.6143,614451.3524,617722.7724,620426.4827
1,753899,2,"Los Angeles, CA",msa,CA,225004.5438,225841.8461,226957.1249,229176.1551,231603.0784,...,893828.1579,885384.4667,874288.3529,864349.4672,861930.4039,865780.1656,873714.4562,885120.3977,898920.6030,912854.6032
2,394463,3,"Chicago, IL",msa,IL,149670.2487,149808.7998,150072.6727,150729.1229,151518.7536,...,287482.3014,286885.9368,286735.4858,287367.7020,289026.7412,291175.5936,293687.0706,296179.3135,298585.6638,300379.1909
3,394514,4,"Dallas, TX",msa,TX,125827.1841,125883.2279,125947.6561,126114.9693,126335.5671,...,375101.6301,372559.5569,370293.4148,368756.1685,368204.4317,368490.4546,369308.9750,370286.4448,371208.7878,371624.0281
4,394692,5,"Houston, TX",msa,TX,120858.3330,120880.8452,120796.4611,120846.9009,120893.4300,...,305224.4451,303461.4198,301787.5004,300640.5033,300192.7671,300347.4117,301151.1126,302072.6358,302946.5474,303302.7017


## 1. Handling Outliers (Excessively Large/Small Values)

In [17]:
# Price columns
date_columns = price.columns[5:]

# Check Excessively Large/Small Values
stats = price[date_columns].describe()

# Max & Min value in each columns
max_per_column = stats.loc['max']
min_per_column = stats.loc['min']

# Check Max & Min value of dataframe
overall_max = max_per_column.max()
overall_min = min_per_column.min()

print("Max value in DataFrame: ", overall_max)
print("Min value in DataFrame: ", overall_min)

Max value in DataFrame:  999999999.0
Min value in DataFrame:  0.0


In [5]:
# 2. 确保日期列的数据类型为数值类型
price[date_columns] = price[date_columns].apply(pd.to_numeric, errors='coerce')

# 3. Thresholds
upper_threshold = 2_000_000
lower_threshold = 10_000

def replace_outliers(row, date_cols, upper_thresh, lower_thresh):

    values = row[date_cols].values
    n = len(values)
    
    # 标记异常值：大于上限或小于下限
    outliers = (values > upper_thresh) | (values < lower_thresh)
    
    # 初始化替换值列表
    replacement_values = values.copy()
    
    i = 0
    while i < n:
        if outliers[i]:
            # 记录连续异常值的起始和结束位置
            start = i
            while i < n and outliers[i]:
                i += 1
            end = i - 1
            
            # 找到左邻居
            left = start - 1
            while left >= 0 and outliers[left]:
                left -= 1
            left_value = values[left] if left >= 0 else np.nan
            
            # 找到右邻居
            right = end + 1
            while right < n and outliers[right]:
                right += 1
            right_value = values[right] if right < n else np.nan
            
            # 计算替换值
            if not np.isnan(left_value) and not np.isnan(right_value):
                replacement = (left_value + right_value) / 2
            elif not np.isnan(left_value):
                replacement = left_value
            elif not np.isnan(right_value):
                replacement = right_value
            else:
                replacement = np.nan  # 如果左右邻居都不存在非异常值，设为 NaN
            
            # 替换连续异常值
            replacement_values[start:end+1] = replacement
        else:
            i += 1
    
    # 返回替换后的值
    return pd.Series(replacement_values, index=date_cols)

# 4. 应用替换函数到每一行
price[date_columns] = price.apply(
    lambda row: replace_outliers(row, date_columns, upper_threshold, lower_threshold),
    axis=1
)

# 5. 验证替换结果
stats = price[date_columns].describe()

# 提取每个日期列的最大值和最小值
max_per_column = stats.loc['max']
min_per_column = stats.loc['min']

# 计算整个 DataFrame 的最大值和最小值
overall_max = max_per_column.max()
overall_min = min_per_column.min()

print("整个 DataFrame 的最大值:", overall_max)
print("整个 DataFrame 的最小值:", overall_min)

# 检查是否存在仍大于上限或小于下限的值
if overall_max > upper_threshold or overall_min < lower_threshold:
    print("警告: 仍存在大于上限阈值或小于下限阈值的值。")
else:
    print("所有异常值均已成功替换，最大值不超过 2,000,000，最小值不低于 10,000。")

# 6. （可选）统计每列中异常值的数量
upper_outliers = (price[date_columns] > upper_threshold).sum()
lower_outliers = (price[date_columns] < lower_threshold).sum()

print("\n每个日期列中大于上限阈值的异常值数量：")
print(upper_outliers)

print("\n每个日期列中小于下限阈值的异常值数量：")
print(lower_outliers)

# 7. （可选）进一步处理替换过程中产生的 NaN 值

# 使用线性插值填补缺失值，沿着列方向（axis=1）
price[date_columns] = price[date_columns].interpolate(method='linear', axis=1, limit_direction='both')

# 使用后向填充填补起始缺失值
price[date_columns] = price[date_columns].fillna(method='bfill', axis=1)

# 使用前向填充填补结束缺失值
price[date_columns] = price[date_columns].fillna(method='ffill', axis=1)

# 使用每行的均值填补剩余的缺失值
price[date_columns] = price[date_columns].apply(
    lambda row: row.fillna(row.mean()), axis=1
)

# 最终检查是否还有缺失值
missing_after_all = price[date_columns].isna().sum().sum()
if missing_after_all > 0:
    print(f"仍有 {missing_after_all} 个缺失值未被替换。")
    # 根据需要选择进一步的填补策略
else:
    print("所有异常值和缺失值均已成功替换。")

/tmp/ipykernel_20867/2005917448.py:34: RuntimeWarning: invalid value encountered in greater
  outliers = (values > upper_thresh) | (values < lower_thresh)
/tmp/ipykernel_20867/2005917448.py:34: RuntimeWarning: invalid value encountered in less
  outliers = (values > upper_thresh) | (values < lower_thresh)


整个 DataFrame 的最大值: 1610331.33
整个 DataFrame 的最小值: 10761.2424
所有异常值均已成功替换，最大值不超过 2,000,000，最小值不低于 10,000。

每个日期列中大于上限阈值的异常值数量：
1/31/2000    0
2/29/2000    0
3/31/2000    0
4/30/2000    0
5/31/2000    0
            ..
5/31/2023    0
6/30/2023    0
7/31/2023    0
8/31/2023    0
9/30/2023    0
Length: 285, dtype: int64

每个日期列中小于下限阈值的异常值数量：
1/31/2000    0
2/29/2000    0
3/31/2000    0
4/30/2000    0
5/31/2000    0
            ..
5/31/2023    0
6/30/2023    0
7/31/2023    0
8/31/2023    0
9/30/2023    0
Length: 285, dtype: int64
所有异常值和缺失值均已成功替换。


## Non Numeric data cleaning

In [6]:
# 2. 定义一个函数来检查是否为数值
def is_numeric(x):
    try:
        float(x)
        return True
    except (ValueError, TypeError):
        return False

# 3. 应用函数到 date_columns，创建一个布尔 DataFrame，True 表示数值，False 表示非数值
numeric_mask = price[date_columns].applymap(is_numeric)

# 4. 标记非数值单元格
non_numeric_mask = ~numeric_mask

# 5. 提取非数值单元格的位置和对应的值
# 使用 stack 将 DataFrame 转换为 Series，方便筛选
non_numeric_locations = non_numeric_mask.stack()

# 过滤出非数值为 True 的位置
non_numeric_locations = non_numeric_locations[non_numeric_locations]

# 获取对应的行索引和列名
non_numeric_indices = non_numeric_locations.index.tolist()

# 打印所有非数值单元格的位置及其值
print("非数值单元格的位置及其值：")
for row, col in non_numeric_indices:
    cell_value = price.at[row, col]
    print(f"行索引: {row}, 列名: '{col}', 值: {cell_value}")

# 6. （可选）统计每列中非数值单元格的数量
non_numeric_counts = non_numeric_mask.sum()

print("\n每个日期列中非数值单元格的数量：")
print(non_numeric_counts)

非数值单元格的位置及其值：

每个日期列中非数值单元格的数量：
1/31/2000    0
2/29/2000    0
3/31/2000    0
4/30/2000    0
5/31/2000    0
            ..
5/31/2023    0
6/30/2023    0
7/31/2023    0
8/31/2023    0
9/30/2023    0
Length: 285, dtype: int64


In [10]:
# 确保日期列的数据类型为数值类型
price[date_columns] = price[date_columns].apply(pd.to_numeric, errors='coerce')

# 使用线性插值方法填补缺失值，沿着列方向（axis=1）
price[date_columns] = price[date_columns].interpolate(method='linear', axis=1, limit_direction='both')

# 验证是否还有缺失值
missing_after_interpolation = price[date_columns].isna().sum().sum()
print(f"插值后剩余缺失值数量: {missing_after_interpolation}")

if missing_after_interpolation > 0:
    print("存在仍未填补的缺失值，进行进一步处理。")
else:
    print("所有缺失值均已通过插值方法成功填补。")

插值后剩余缺失值数量: 0
所有缺失值均已通过插值方法成功填补。


In [13]:
price.to_csv("Data_cleaned.csv",index=0)

## Modeling Process

Convert the data into a format suitable for modeling

In [ ]:
import pandas as pd

# 读取Excel文件
df = pd.read_csv('Data_cleaned.csv')

# 查看数据列名
print(df.columns)

# 选择从F列开始的房价列
price_columns = df.columns[5:]  

for column in price_columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# 将数据转换为长格式
df_long = df.melt(id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
                  value_vars=df.columns[5:],  # 选择从第6列开始的月份列
                  var_name='Date',
                  value_name='Price')

# 将 'Date' 列转换为 datetime 类型
df_long['Date'] = pd.to_datetime(df_long['Date'], format='%m/%d/%Y')

# 按 RegionName 分组
df_long.set_index('Date', inplace=True)

Get predicted value

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 假设 df_long 是您的原始数据框
# 创建一个新的 DataFrame 用来保存预测结果
df_forecasted = df_long.copy()

# 获取所有唯一的地区名称
regions = df_long['RegionName'].unique()

# 创建一个列表来保存预测结果
forecast_list = []

# 遍历所有地区进行预测
for region in regions:
    print(region)
    region_data = df_long[df_long['RegionName'] == region]
    region_data = region_data.dropna()
    
    # 只保留 'Price' 列作为时间序列数据
    region_data = region_data[['Price']]
    
    # 使用SARIMAX模型进行建模
    model = SARIMAX(region_data, 
                    order=(0, 1, 0),  # 非季节性部分 (p, d, q)
                    seasonal_order=(1, 1, 0, 12))  # 季节性部分 (P, D, Q, S)
    model_fit = model.fit()
    
    # 预测未来5个月的价格
    forecast_steps = 5
    forecast = model_fit.forecast(steps=forecast_steps)
    
    # 创建新的列名，例如 'Forecast_Price'
    forecast_column = f'{region}'
    
    # 将预测值填入 df_forecasted 中，扩展预测数据到 df_forecasted
    #forecast_index = pd.date_range(start=region_data.index[-1] + pd.Timedelta(days=1), 
                                   #periods=forecast_steps, freq='M')

    forecast_index = ['2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29']
    forecast_index = pd.to_datetime(forecast_index)
    forecast_series = pd.Series(forecast.values, index=forecast_index, name=forecast_column)
    
    # 将 Series 转换为 DataFrame
    df = forecast_series.to_frame()
    
    # 重命名列
    df.columns = ['Price']
    
    # 添加 RegionName 列，所有行都为 'Ludington, MI'
    df['RegionName'] = forecast_series.name
    
    # 重置索引，并将索引列名更改为 'Date'
    df = df.reset_index()
    df.rename(columns={'index': 'Date'}, inplace=True)
    
    # 按照期望的格式排列列
    df = df[['Date', 'RegionName', 'Price']]
    df.set_index('Date', inplace=True)
    print(df)

    forecast_list.append(df)

df_combined = pd.concat(forecast_list, ignore_index=False)

Combine predicted value into orginial dataframe

In [ ]:
df = pd.read_csv('/Users/yzhang3/Desktop/Data_cleaned.csv')
# 将 df2 的列名中的日期部分转换为标准格式
date_columns = ['10/31/2023', '11/30/2023', '12/31/2023', '01/31/2024', '02/29/2024']
formatted_dates = pd.to_datetime(date_columns)

print(date_columns, formatted_dates)

# 遍历 df2 的每一行
for i, row in df.iterrows():
    # 获取当前行的 RegionName
    region_name = row['RegionName']
    
    # 遍历每个日期列
    for date, col in zip(formatted_dates, date_columns):
        # 检查 df1 中是否有对应的日期和 RegionName
        if date in df_combined.index and not df_combined[df_combined['RegionName'] == region_name].empty:
            # 获取价格
            matching_price = df_combined.loc[(df_combined.index == date) & (df_combined['RegionName'] == region_name), 'Price']
            if not matching_price.empty:
                # 填充到 df2 的对应列
                df.at[i, col] = matching_price.values[0]

In [ ]:
df.to_csv('Data_with_predict.csv')